# W_LEDGER_DS ETL Process
### Oracle EBS GL_LEDGERS to EDW W_LEDGER_DS
**ODI Package:** SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION  
**LKM:** BIAPPS SQL to Oracle (Multi Transport)  
**IKM:** BIAPPS Oracle Control Append

In [ ]:
%sql
-- Step 1: Variable IS_INCREMENTAL - Check if incremental load is enabled
CREATE OR REPLACE TEMP VIEW VAR_IS_INCREMENTAL AS
SELECT 
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION'
  AND (DATASOURCE_NUM_ID = 380 OR DATASOURCE_NUM_ID = 380)
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1'

In [ ]:
%sql
-- Step 2: Variable LAST_EXTRACT_DATE - Get last extraction date
CREATE OR REPLACE TEMP VIEW VAR_LAST_EXTRACT_DATE AS
SELECT
  COALESCE(
    date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'),
    '1970-01-01 00:00:00'
  ) AS LAST_EXTRACT_DATE
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION'
  AND DATASOURCE_NUM_ID = 380
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 3: Drop and Create Work Table C$_2624261_2_0 (as temp view)
DROP VIEW IF EXISTS C$_2624261_2_0;

CREATE OR REPLACE TEMP VIEW C$_2624261_2_0 AS
SELECT
  SQ_GL_LEDGERS.NAME AS NAME,
  SQ_GL_LEDGERS.SHORT_NAME AS SHORT_NAME,
  SQ_GL_LEDGERS.DESCRIPTION AS DESCRIPTION,
  SQ_GL_LEDGERS.PERIOD_SET_NAME AS PERIOD_SET_NAME,
  SQ_GL_LEDGERS.LAST_UPDATE_DATE AS LAST_UPDATE_DATE,
  SQ_GL_LEDGERS.CREATION_DATE AS CREATION_DATE,
  SQ_GL_LEDGERS.USER_PERIOD_TYPE AS USER_PERIOD_TYPE,
  CAST(SQ_GL_LEDGERS.CHART_OF_ACCOUNTS_ID AS STRING) AS CHART_OF_ACCOUNTS,
  COALESCE(SQ_GL_LEDGERS.CURRENCY_CODE, '__UNASSIGNED__') AS CURRENCY_CODE,
  COALESCE(SQ_GL_LEDGERS.SLA_ACCOUNTING_METHOD_CODE, '__UNASSIGNED__') AS SLA_ACCOUNTING_METHOD_CODE,
  COALESCE(SQ_GL_LEDGERS.LEDGER_CATEGORY_CODE, '__UNASSIGNED__') AS LEDGER_CATEGORY_CODE,
  CAST(SQ_GL_LEDGERS.CREATED_BY AS STRING) AS CREATED_BY_ID,
  CAST(SQ_GL_LEDGERS.LAST_UPDATED_BY AS STRING) AS CHANGED_BY_ID,
  CAST(SQ_GL_LEDGERS.LEDGER_ID AS STRING) AS INTEGRATION_ID
FROM (
  SELECT 
    GL_LEDGERS.LEDGER_ID,
    GL_LEDGERS.NAME,
    GL_LEDGERS.SHORT_NAME,
    GL_LEDGERS.DESCRIPTION,
    GL_LEDGERS.LEDGER_CATEGORY_CODE,
    GL_LEDGERS.CHART_OF_ACCOUNTS_ID,
    GL_LEDGERS.CURRENCY_CODE,
    GL_LEDGERS.PERIOD_SET_NAME,
    GL_LEDGERS.SLA_ACCOUNTING_METHOD_CODE,
    GL_LEDGERS.LAST_UPDATE_DATE,
    GL_LEDGERS.LAST_UPDATED_BY,
    GL_LEDGERS.CREATION_DATE,
    GL_LEDGERS.CREATED_BY,
    GL_PERIOD_TYPES.USER_PERIOD_TYPE
  FROM workspace.oracle_ebs.GL_LEDGERS GL_LEDGERS
  INNER JOIN workspace.oracle_ebs.GL_PERIOD_TYPES GL_PERIOD_TYPES
    ON GL_LEDGERS.ACCOUNTED_PERIOD_TYPE = GL_PERIOD_TYPES.PERIOD_TYPE
  LEFT JOIN workspace.oracle_ebs.XLA_ACCTG_METHODS_TL XLA_ACCTG_METHODS_TL
    ON GL_LEDGERS.SLA_ACCOUNTING_METHOD_TYPE = XLA_ACCTG_METHODS_TL.ACCOUNTING_METHOD_TYPE_CODE
    AND GL_LEDGERS.SLA_ACCOUNTING_METHOD_CODE = XLA_ACCTG_METHODS_TL.ACCOUNTING_METHOD_CODE
    AND XLA_ACCTG_METHODS_TL.LANGUAGE = 'US'
  WHERE COALESCE(GL_LEDGERS.COMPLETE_FLAG, 'Y') = 'Y'
    AND GL_LEDGERS.OBJECT_TYPE_CODE = 'L'
) SQ_GL_LEDGERS

In [ ]:
%sql
-- Step 4: Insert new rows into W_LEDGER_DS from work table
INSERT INTO workspace.oracle_edw.W_LEDGER_DS
(
  LEDGER_NAME,
  LEDGER_SHORT_NAME,
  LEDGER_DESC,
  CHART_OF_ACCOUNTS,
  CURRENCY_CODE,
  CALENDER_NAME,
  SLA_ACCOUNTING_METHOD_CODE,
  PERIOD_TYPE,
  LEDGER_CATEGORY_CODE,
  CREATED_BY_ID,
  CHANGED_BY_ID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  DELETE_FLG,
  TENANT_ID
)
SELECT
  NAME AS LEDGER_NAME,
  SHORT_NAME AS LEDGER_SHORT_NAME,
  DESCRIPTION AS LEDGER_DESC,
  CHART_OF_ACCOUNTS,
  CURRENCY_CODE,
  PERIOD_SET_NAME AS CALENDER_NAME,
  SLA_ACCOUNTING_METHOD_CODE,
  USER_PERIOD_TYPE AS PERIOD_TYPE,
  LEDGER_CATEGORY_CODE,
  CREATED_BY_ID,
  CHANGED_BY_ID,
  CAST(CREATION_DATE AS TIMESTAMP) AS CREATED_ON_DT,
  CAST(LAST_UPDATE_DATE AS TIMESTAMP) AS CHANGED_ON_DT,
  380 AS DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  'N' AS DELETE_FLG,
  '1' AS TENANT_ID
FROM C$_2624261_2_0

In [ ]:
%sql
-- Step 5: Update reference dates in W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.W_ETL_LOAD_DATES AS TGT
USING (
  SELECT
    380 AS DATASOURCE_NUM_ID,
    'SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION' AS PACKAGE_NAME,
    'W_LEDGER_DS' AS TARGET_TABLE_NAME,
    '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
    2624261 AS ETL_PROC_WID,
    12345 AS LOAD_PLAN_ID,
    date_add(current_timestamp(), -30) AS WIP_LOAD_START_DATE,
    current_timestamp() AS ETL_LOAD_DATE,
    (SELECT CASE WHEN IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM VAR_IS_INCREMENTAL) AS COMMITTED
) SRC
ON TGT.DATASOURCE_NUM_ID = SRC.DATASOURCE_NUM_ID
  AND TGT.PACKAGE_NAME = SRC.PACKAGE_NAME
  AND TGT.ETL_USAGE_CODE = SRC.ETL_USAGE_CODE
WHEN MATCHED THEN
  UPDATE SET
    TGT.TARGET_TABLE_NAME = SRC.TARGET_TABLE_NAME,
    TGT.ETL_PROC_WID = SRC.ETL_PROC_WID,
    TGT.LOAD_PLAN_ID = SRC.LOAD_PLAN_ID,
    TGT.WIP_LOAD_START_DATE = SRC.WIP_LOAD_START_DATE,
    TGT.ETL_LOAD_DATE = SRC.ETL_LOAD_DATE,
    TGT.COMMITTED = SRC.COMMITTED
WHEN NOT MATCHED THEN
  INSERT (
    DATASOURCE_NUM_ID,
    PACKAGE_NAME,
    TARGET_TABLE_NAME,
    ETL_USAGE_CODE,
    ETL_PROC_WID,
    LOAD_PLAN_ID,
    WIP_LOAD_START_DATE,
    ETL_LOAD_DATE,
    COMMITTED
  )
  VALUES (
    SRC.DATASOURCE_NUM_ID,
    SRC.PACKAGE_NAME,
    SRC.TARGET_TABLE_NAME,
    SRC.ETL_USAGE_CODE,
    SRC.ETL_PROC_WID,
    SRC.LOAD_PLAN_ID,
    SRC.WIP_LOAD_START_DATE,
    SRC.ETL_LOAD_DATE,
    SRC.COMMITTED
  )

In [ ]:
%sql
-- Step 6: Insert reference dates to History Table W_ETL_LOAD_DATES_LOG
INSERT INTO workspace.oracle_edw.W_ETL_LOAD_DATES_LOG
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  2624261 AS SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE DATASOURCE_NUM_ID = 380
  AND PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_LEDGERDIMENSION'
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 7: Drop work table (temp view)
DROP VIEW IF EXISTS C$_2624261_2_0;
DROP VIEW IF EXISTS VAR_IS_INCREMENTAL;
DROP VIEW IF EXISTS VAR_LAST_EXTRACT_DATE